In [7]:
from azureml.core import Workspace, Datastore, Dataset
import pandas as pd

Livedoor ニュースのコーパスをダウンロードして利用します。

In [1]:
from urllib.request import urlretrieve
import tarfile

text_url = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
file_path = "../data/raw/ldcc-20140209.tar.gz"
urlretrieve(text_url, file_path)

('../data/raw/ldcc-20140209.tar.gz',
 <http.client.HTTPMessage at 0x7fe4fc597e48>)

In [3]:
# gz ファイルを解凍します。そこそこ時間がかかります。
with tarfile.open('../data/raw/ldcc-20140209.tar.gz', 'r:gz') as tar:
    tar.extractall(path='../data/processed/livedoor')
    tar.close()

Pandas DataFrame へのロード

In [8]:
columns = ['url', 'date', 'label', 'title', 'text']
df = pd.DataFrame(columns = columns)
#df.set_index('url',inplace=True)

In [10]:
path = "../data/processed/livedoor/text"


In [11]:
for folder_name in os.listdir(path):
    print(folder_name)
    if folder_name.endswith(".txt") :
        continue
    for file in os.listdir(os.path.join(path, folder_name)):
        if folder_name == "LICENSE.txt" :
            continue
        with open(os.path.join(path, folder_name, file), 'r') as f:
            lines = f.read().split('\n')
            if len(lines) == 1:
                continue
            url = lines[0]
            date = lines[1]
            label = folder_name
            title = lines[3]
            text = "".join(lines[4:])
            data = {'url': url, 'date':date, 'label': label, 'title':title, 'text':text}
        s = pd.Series(data)        
        df = df.append(s, ignore_index=True)


CHANGES.txt
dokujo-tsushin
it-life-hack
kaden-channel
livedoor-homme
movie-enter
peachy
README.txt
smax
sports-watch
topic-news


In [12]:
df.to_csv("../data/processed/livedoor-corpus.csv", index=False)


## Azure Machine Learning ワークスペース接続

In [14]:
ws = Workspace.from_config()
print(ws)

Workspace.create(name='azureml_text', subscription_id='9c0f91b8-eb2f-484c-979c-15848c098a6b', resource_group='azureml_text')


In [15]:
datastore = ws.get_default_datastore()
datastore.upload_files(files=['../data/processed/livedoor-corpus.csv'],
                       target_path='livedoor-corpus',
                       overwrite=True,
                       show_progress=False)

$AZUREML_DATAREFERENCE_8464dd6d342b4ffdbc8061e39cb59f1e

In [16]:
datastore_path = [(datastore, 'livedoor-corpus/livedoor-corpus.csv')]


In [17]:
livedoor_ds = Dataset.Tabular.from_delimited_files(path=datastore_path)


In [18]:
livedoor_ds.register(workspace=ws, name='livedoor',description='livedoor corpus', create_new_version = True)


{
  "source": [
    "('workspaceblobstore', 'livedoor-corpus/livedoor-corpus.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "38aa3b80-d633-4a05-ab9a-186a3ce51ffc",
    "name": "livedoor",
    "version": 2,
    "description": "livedoor corpus",
    "workspace": "Workspace.create(name='azureml_text', subscription_id='9c0f91b8-eb2f-484c-979c-15848c098a6b', resource_group='azureml_text')"
  }
}

In [19]:
dataset = Dataset.get_by_name(ws, name='livedoor')
df = dataset.to_pandas_dataframe()

In [20]:
# 欠損値の除外
df = df[pd.isna(df["text"])==False]
df.head()


,url,date,label,title,text
0,http://news.livedoor.com/article/detail/4778030/,2010-05-22 05:30:00,dokujo-tsushin,もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり…...,「お願いがあるんだけど……友人代表のスピーチ、やってくれないかな？」 さてそんなとき、独女...
1,http://news.livedoor.com/article/detail/4778031/,2010-05-21 05:30:00,dokujo-tsushin,携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だった。恋人と別れたら、手帳に書...,「5年前に別れた彼からメールが届いてビックリしました」とは尚美さん（36歳） 「彼の浮気が...
2,http://news.livedoor.com/article/detail/4782522/,2010-05-23 02:00:00,dokujo-tsushin,「男性はやっぱり、女性の“すっぴん”が大好きなんですかね」と不満そうに話すのは、出版関係で...,今年に入ってから、“すっぴん”をブログで披露した芸能人は、小倉優子、安倍なつみ、モーニング...
3,http://news.livedoor.com/article/detail/4788357/,2010-05-25 05:00:00,dokujo-tsushin,ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「そげる→たわむ→外に流れ...,これは、４月に開催されたワコール人間科学研究所の記者発表「からだのエイジング（加齢による体...
4,http://news.livedoor.com/article/detail/4788362/,2010-05-26 05:00:00,dokujo-tsushin,6月から支給される子ども手当だが、当初は子ども一人当たり月額2万6000円が支給されるはず...,先日は在日外国人男性がタイで養子縁組をしたと称する554人分の子ども手当を申請しようとして...
